In [1]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(
    f"pip install -qq"
    f" ipykernel=={get_distribution('ipykernel').version}"
    f" ipython=={get_distribution('ipython').version}"
    f" tornado=={get_distribution('tornado').version}"
    f" oct2py"
)

# install packages
!pip install -qq matpower matpowercaseframesa

In [2]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.7.2
matpower version: 8.0.0.2.2.2


In [3]:
import os

from matpower import path_matpower, start_instance
from matpowercaseframes import CaseFrames

## Inspect data

In [4]:
path = os.path.join(path_matpower, "data/case9.m")

In [5]:
m = start_instance()

In [6]:
cf = CaseFrames(path, load_case_engine=m)
cf.infer_numpy()  # for better print, convert floats to ints
cf.branch

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX
1,1,4,0.0000,0.0576,0.000,250,250,250,0,0,1,-360,360
2,4,5,0.0170,0.0920,0.158,250,250,250,0,0,1,-360,360
3,5,6,0.0390,0.1700,0.358,150,150,150,0,0,1,-360,360
4,3,6,0.0000,0.0586,0.000,300,300,300,0,0,1,-360,360
5,6,7,0.0119,0.1008,0.209,150,150,150,0,0,1,-360,360
6,7,8,0.0085,0.0720,0.149,250,250,250,0,0,1,-360,360
7,8,2,0.0000,0.0625,0.000,250,250,250,0,0,1,-360,360
8,8,9,0.0320,0.1610,0.306,250,250,250,0,0,1,-360,360
9,9,4,0.0100,0.0850,0.176,250,250,250,0,0,1,-360,360


## Run base

In [7]:
mpc = m.runpf(cf.to_mpc(), verbose=True)
cf_sol = CaseFrames(mpc)
cf_sol.infer_numpy()  # for better print, convert floats to ints


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful

Converged in 0.12 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers    

## Copy and editing branch

In [8]:
new_branch = cf_sol.branch.loc[2, :].copy()  # copy cf_sol.branch.index 2
new_branch["T_BUS"] = 8  # connect to bus 8
new_branch.rename(10, inplace=True)  # give it name branch.index 10
cf_sol.branch = cf_sol.branch._append([new_branch])

In [9]:
cf_sol.branch

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX,PF,QF,PT,QT
1,1.0,4.0,0.0000,0.0576,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,71.641021,27.045924,-71.641021,-23.923127
2,4.0,5.0,0.0170,0.0920,0.158,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,30.703670,1.030006,-30.537263,-16.543365
3,5.0,6.0,0.0390,0.1700,0.358,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0,-59.462737,-13.456635,60.816586,-18.074836
4,3.0,6.0,0.0000,0.0586,0.000,300.0,300.0,300.0,0.0,0.0,1.0,-360.0,360.0,85.000000,-10.859709,-85.000000,14.955327
5,6.0,7.0,0.0119,0.1008,0.209,150.0,150.0,150.0,0.0,0.0,1.0,-360.0,360.0,24.183414,3.119508,-24.095417,-24.295823
6,7.0,8.0,0.0085,0.0720,0.149,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,-75.904583,-10.704177,76.379866,-0.797331
7,8.0,2.0,0.0000,0.0625,0.000,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,-163.000000,9.178149,163.000000,6.653660
8,8.0,9.0,0.0320,0.1610,0.306,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,86.620134,-8.380817,-84.320163,-11.312751
9,9.0,4.0,0.0100,0.0850,0.176,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,-40.679837,-38.687249,40.937352,22.893121
10,4.0,8.0,0.0170,0.0920,0.158,250.0,250.0,250.0,0.0,0.0,1.0,-360.0,360.0,30.703670,1.030006,-30.537263,-16.543365


In [10]:
mpc = m.runpf(cf_sol.to_mpc(), verbose=True)
cf_sol = CaseFrames(mpc)
cf_sol.infer_numpy()  # for better print, convert floats to ints


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 3 iterations.
PF successful

Converged in 0.09 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    318.5              -0.5
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches          10     Losses (I^2 * Z)         3.46             43.28
Transformers    

In [11]:
cf_sol.branch

,F_BUS,T_BUS,BR_R,BR_X,BR_B,RATE_A,RATE_B,RATE_C,TAP,SHIFT,BR_STATUS,ANGMIN,ANGMAX,PF,QF,PT,QT
1,1,4,0.0000,0.0576,0.000,250,250,250,0,0,1,-360,360,70.457046,17.714693,-70.457046,-14.903921
2,4,5,0.0170,0.0920,0.158,250,250,250,0,0,1,-360,360,45.777118,-1.361320,-45.434013,-13.363041
3,5,6,0.0390,0.1700,0.358,150,150,150,0,0,1,-360,360,-44.565986,-16.636959,45.314981,-17.800140
4,3,6,0.0000,0.0586,0.000,300,300,300,0,0,1,-360,360,85.000000,-14.614177,-85.000000,18.763140
5,6,7,0.0119,0.1008,0.209,150,150,150,0,0,1,-360,360,39.685019,-0.963000,-39.498282,-19.524948
6,7,8,0.0085,0.0720,0.149,250,250,250,0,0,1,-360,360,-60.501718,-15.475052,60.805243,2.350622
7,8,2,0.0000,0.0625,0.000,250,250,250,0,0,1,-360,360,-163.000000,19.424015,163.000000,-3.610786
8,8,9,0.0320,0.1610,0.306,250,250,250,0,0,1,-360,360,59.663821,-7.526282,-58.571139,-18.690375
9,9,4,0.0100,0.0850,0.176,250,250,250,0,0,1,-360,360,-66.428861,-31.309625,66.916694,17.234793
10,4,8,0.0170,0.0920,0.158,250,250,250,0,0,1,-360,360,-42.236766,-0.969552,42.530936,-14.248355
